In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('/content/first_false_demo.csv')
df2 = pd.read_csv('/content/second_false_demo.csv')

In [3]:
df1.shape

(56254, 8)

In [4]:
df2.shape

(306377, 10)

In [5]:
df = pd.concat([df1,df2])

In [6]:
df.shape

(362631, 10)

In [7]:
df.head()

,Unnamed: 0,date,state,district,pincode,demo_age_5_17,demo_age_17_,match,Unnamed: 0.1,match_district
0,20,01-03-2025,tamil nadu,kancheepuram,603202,42,700,False,NaN,NaN
1,49,01-03-2025,uttar pradesh,saharanpur,247670,10,105,False,NaN,NaN
2,57,01-03-2025,gujarat,chhotaudepur,391155,26,69,False,NaN,NaN
3,74,01-03-2025,chandigarh,chandigarh,160103,12,78,False,NaN,NaN
4,105,01-03-2025,gujarat,mahesana,384212,27,151,False,NaN,NaN


In [10]:
# df.drop(['match','match_district'],axis=1,inplace=True)
df.drop(['Unnamed: 0.1','Unnamed: 0'],axis=1,inplace=True)

In [11]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,01-03-2025,tamil nadu,kancheepuram,603202,42,700
1,01-03-2025,uttar pradesh,saharanpur,247670,10,105
2,01-03-2025,gujarat,chhotaudepur,391155,26,69
3,01-03-2025,chandigarh,chandigarh,160103,12,78
4,01-03-2025,gujarat,mahesana,384212,27,151


In [12]:
# Checking
import pandas as pd
import json

# -----------------------------
# 1. Load JSON (true data)
# -----------------------------
with open("/content/pin-code-data.json", "r") as f:
    pin_data = json.load(f)

# -----------------------------
# 2. Helper: clean text
# -----------------------------
def clean_text(x):
    return str(x).strip().upper()

# -----------------------------
# 3. Build DISTRICT → PINCODES mapping
# -----------------------------
district_to_pins = {}

for pin, info in pin_data.items():
    district = clean_text(info["district"])
    district_to_pins.setdefault(district, []).append(int(pin))

# -----------------------------
# 4. Validation + correction logic
# -----------------------------
def check_and_fix(row):
    df_district = clean_text(row["district"])
    df_pin = row["pincode"]

    # District not found in JSON → cannot validate
    if df_district not in district_to_pins:
        return df_pin, "unknown"

    valid_pins = district_to_pins[df_district]

    # Pincode correct
    if df_pin in valid_pins:
        return df_pin, "yes"

    # Pincode wrong → replace
    return valid_pins[0], "no"


# -----------------------------
# 5. Apply logic
# -----------------------------
df[["pincode", "same"]] = df.apply(
    lambda r: pd.Series(check_and_fix(r)),
    axis=1
)

# -----------------------------
# 6. Result
# -----------------------------
print(df.head())


         date          state      district  pincode  demo_age_5_17  \
0  01-03-2025     tamil nadu  kancheepuram   603202             42   
1  01-03-2025  uttar pradesh    saharanpur   247001             10   
2  01-03-2025        gujarat  chhotaudepur   391125             26   
3  01-03-2025     chandigarh    chandigarh   160101             12   
4  01-03-2025        gujarat      mahesana   384001             27   

   demo_age_17_     same  
0           700  unknown  
1           105       no  
2            69       no  
3            78       no  
4           151       no  


In [13]:
df[df['same']=='unknown'].shape

(135456, 7)

In [14]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,same
0,01-03-2025,tamil nadu,kancheepuram,603202,42,700,unknown
1,01-03-2025,uttar pradesh,saharanpur,247001,10,105,no
2,01-03-2025,gujarat,chhotaudepur,391125,26,69,no
3,01-03-2025,chandigarh,chandigarh,160101,12,78,no
4,01-03-2025,gujarat,mahesana,384001,27,151,no


In [15]:
df['pincode'].isna().sum()

np.int64(0)

In [ ]:
df.to_csv('check_bio.csv',index=False)

In [16]:
import numpy as np

df["not found"] = np.where(
    df["same"] == "unknown",
    df["district"],
    np.nan
)


In [17]:
df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_,same,not found
0,01-03-2025,tamil nadu,kancheepuram,603202,42,700,unknown,kancheepuram
1,01-03-2025,uttar pradesh,saharanpur,247001,10,105,no,NaN
2,01-03-2025,gujarat,chhotaudepur,391125,26,69,no,NaN
3,01-03-2025,chandigarh,chandigarh,160101,12,78,no,NaN
4,01-03-2025,gujarat,mahesana,384001,27,151,no,NaN


In [20]:
df[~df['not found'].isna()]['not found'].unique()

array(['kancheepuram', 'ananthapuramu', 'puducherry', 'ahmedabad',
       'south 24 parganas', 'uttar bastar kanker', 'kamle', 'bara banki',
       'cooch behar', 'dakshin dinajpur', 'malda', 'north 24 parganas',
       'paschim medinipur', 'purba medinipur', 'uttar dinajpur',
       'viluppuram', 'mohla-manpur-ambagarh chouki',
       'dakshin bastar dantewada', 'angul', 'udham singh nagar',
       'east singhbhum', 'yanam', 'narayanpur', 'sant kabir nagar',
       'lohardaga', 'saitual', 'sri potti sriramulu nellore',
       'davanagere', 'leh', 'didwana-kuchaman', 'narsimhapur',
       'khairthal-tijara', 'thoothukkudi', 'ahilyanagar', 'visakhapatnam',
       'siddharthnagar', 'dahod', 'jajpur', 'maihar', 'kabeerdham',
       'shrawasti', 'narmadapuram', 'uttarkashi', 'ferozepur',
       'charkhi dadri', 'vijayanagara', 'rudraprayag', 'kushinagar',
       'chhatrapati sambhajinagar', 'gaurela-pendra-marwahi', 'mauganj',
       'chamarajanagar', 'pandhurna', 'dharashiv', 'sakti', 'de